### CPS ASEC replicate Census Median HH Income Estimates

Brian Dew, brian.w.dew@gmail.com

January 15, 2019

----

Try to replicate the median household income statistics [published](https://www.census.gov/library/publications/2018/demo/p60-263.html) by Census, using a binned- and weighted-median.

The number I want to get (at least very close) is $61,372.


Also want to clean up the code a bit.

In [3]:
# Import relevant libraries (python 3.6)
import os, re, struct
import pandas as pd
import numpy as np

import wquantiles

os.chdir('/home/brian/Documents/ASEC/data/')

In [10]:
# read data dictionary text file 
pubuse_file = 'asec2018_pubuse.dat'
dd_file = '08ASEC2018_Data_Dict_Full.txt'
data_dict = open(dd_file, 'r', encoding='iso-8859-1').read()

In [11]:
# Retrieve column info from dictionary
p = re.compile('D (\w+)\s+(\d{1,2})\s+(\d+)\s+')
var_key = pd.DataFrame(p.findall(data_dict), columns=['Var', 'Len', 'Loc'])
var_key = var_key.apply(pd.to_numeric, errors='ignore')

# Filter out columns of interest
s = ['H_HHTYPE', 'H_SEQ', 'H_TYPE', 'HSUP_WGT', 'HTOTVAL']
s_key = var_key[var_key['Var'].isin(s)]

In [12]:
# Read raw fwf file
data = pd.read_fwf(pubuse_file, header=None, names=list(s_key.Var),# nrows=1000,
                 colspecs=list(zip(s_key.Loc-1, s_key.Loc + s_key.Len-1)))

In [13]:
# Median Household Income (Close)
df = data[data['H_HHTYPE'] == 1]
df = df.drop_duplicates(subset='H_SEQ', keep='first')
df = df[df['H_TYPE'] <= 8]

print(f"Number of Households: {df.HSUP_WGT.sum()/100:,.0f}")
med_inc = wquantiles.median(df['HTOTVAL'], df['HSUP_WGT'])
print(f"2017 Median HH Income: ${med_inc:,.2f}")

Number of Households: 127,586,152
2017 Median HH Income: $60,885.12


In [14]:
df['wage_range'] = pd.cut(df['HTOTVAL'], list(range(-35000,10000000,2500)), include_lowest=True)
df = df.sort_values('HTOTVAL')#.dropna(subset=['wage_range'])
midpt = df['HSUP_WGT'].sum() * 0.5
df['cs'] = df['HSUP_WGT'].cumsum()
print('median: ' + str(df.iloc[(df['cs']-midpt).abs().argsort()[:1]].wage_range.values[0].mid))
print('weighted median: ' + str(wquantiles.median(df['HTOTVAL'], df['HSUP_WGT'])))
n = list(df['wage_range'].unique()).index(df.iloc[(df['cs']-midpt).abs().argsort()[:1]].wage_range.values[0])
lowval = df[df['wage_range'] == list(df['wage_range'].unique())[n-1]].iloc[-1].cs
highval = df[df['wage_range'] == list(df['wage_range'].unique())[n]].iloc[-1].cs
print('binned, weighted median: ', str((((midpt - lowval) / (highval - lowval)) * 50) + df.iloc[(df['cs']-midpt).abs().argsort()[:1]].wage_range.values[0].left))

median: 61250.0
weighted median: 60885.11841403385
binned, weighted median:  60023.512617147855
